In [1]:
import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from tensorflow.keras.preprocessing.text import Tokenizer


In [3]:
import pandas as pd
import re
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load dataset
df = pd.read_csv("/content/Synthetic_Data_10K.csv")

# Drop empty rows
df.dropna(subset=["input", "output"], inplace=True)

# Ensure no empty strings
df = df[df["input"].str.strip() != ""]
df = df[df["output"].str.strip() != ""]

print(f"✅ Cleaned dataset: {df.shape[0]} rows")


from transformers import AutoTokenizer

# Load tokenizer
model_name = "facebook/blenderbot-400M-distill"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# Set padding token
tokenizer.pad_token = tokenizer.eos_token

print("✅ Padding token set:", tokenizer.pad_token)


from torch.utils.data import Dataset

class MentalHealthDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        user_input = str(self.dataframe.iloc[index]['input'])
        bot_response = str(self.dataframe.iloc[index]['output'])

        # Tokenize input
        input_encodings = self.tokenizer(
            user_input, truncation=True, padding="max_length",
            max_length=self.max_length, return_tensors="pt"
        )

        # Tokenize output (labels)
        output_encodings = self.tokenizer(
            bot_response, truncation=True, padding="max_length",
            max_length=self.max_length, return_tensors="pt"
        )

        # ✅ Replace padding tokens in labels with -100
        labels = output_encodings["input_ids"].squeeze(0)
        labels[labels == tokenizer.pad_token_id] = -100  # Mask out padding tokens

        return {
            "input_ids": input_encodings["input_ids"].squeeze(0),
            "attention_mask": input_encodings["attention_mask"].squeeze(0),
            "labels": labels
        }

# ✅ Reload dataset with fixes
mental_health_dataset = MentalHealthDataset(df, tokenizer)
print(f"✅ Fixed Dataset Loaded: {len(mental_health_dataset)} samples")




✅ Cleaned dataset: 9747 rows


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

✅ Padding token set: </s>
✅ Fixed Dataset Loaded: 9747 samples


In [7]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./chatbot_model",  # Model save directory
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=500,
    evaluation_strategy="epoch",  # Evaluate after every epoch
    save_total_limit=2,  # Keep only the last 2 checkpoints
    learning_rate=5e-5,  # Adjust as needed
    weight_decay=0.01,
    push_to_hub=False  # Set to True if using Hugging Face Hub
)


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=mental_health_dataset,
    tokenizer=tokenizer,
)


<ipython-input-8-e8356b14d947>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: You have set `args.eval_strategy` to IntervalStrategy.EPOCH but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 